# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:
        csvreader = csv.reader(csvfile) 
        next(csvreader)      
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

In [6]:
session.execute("""CREATE KEYSPACE IF NOT EXISTS cassandra 
                   WITH REPLICATION =  { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
                """)


In [8]:
session.set_keyspace('cassandra')

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
#As we need artist,song title and song's length as per the sessionid & iteminsession we need to select all these columns when we create a table.
#Create table naming it "Session_event_details" as the query deals with session details and only create if table doesn't exist.
#Sessionid & iteminsession should be considered as the primary key as both of them uniquely identify the rows.
#As part of Partition row store create and instert statments are made in the order of Primary key
# Select statment brings the gives the Required out as per question 1.
query = "CREATE TABLE IF NOT EXISTS cassandra.Session_event_details "
query = query + "(sessionid int,iteminsession int,artist_name text,song_title text,song_length float, PRIMARY KEY (sessionid,iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)


                    

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO cassandra.Session_event_details (sessionid,iteminsession,artist_name,song_title,song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, ( int(line[8]), int(line[3]),line[0], line[9], float(line[5])))

In [12]:
query = "select artist_name,song_title,sessionid,song_length from Session_event_details WHERE sessionid=338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title,row.sessionid,row.song_length)

Faithless Music Matters (Mark Knight Dub) 338 495.30731201171875


In [13]:
#Requirment is to extract  song title, user(first_name and last_name) which is sorted by Iteminsession as per userid and  sessionid. To achieve this we need to select Specified columns when table is created.
#Create table naming it "user_usage_hist" as the query deals with User usage history and only create if table doesn't exist.
# User_id and sessionid should be considered as the primary key as both of them uniquely identify the rows.Where as "Iteminsession" should be considered as ClusterKey as it will sort the results based on Iteminsession.
#As part of Partition row store create and instert statments are made in the order of Primary key & Cluster key
# Select statment gives the Required out as per question 2.
query = "CREATE TABLE IF NOT EXISTS cassandra.User_Usage_hist "
query = query + "(userid int,sessionid int,iteminsession int,first_name text,last_name text, artist_name text,song_title text, PRIMARY KEY ((userid,sessionid),iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                  

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO cassandra.User_Usage_hist (userid,sessionid,iteminsession,first_name,last_name,artist_name,song_title)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))

In [15]:
query = "select first_name,last_name, artist_name,song_title from User_Usage_hist WHERE userid=10 and sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name,row.artist_name,row.song_title)

Sylvie Cruz Down To The Bone Keep On Keepin' On
Sylvie Cruz Three Drives Greece 2000
Sylvie Cruz Sebastien Tellier Kilometer
Sylvie Cruz Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


In [16]:
#As we need user_name(first and last name )who listned to the song 'All Hands Against his own',we need to select all these columns when we create a table.We also need to select user_id as it gives us the unique user. First and last name are unique. 
#Create table naming it "user_song" as the query deals with song user listening and only create if table doesn't exist.
#Song_title and User_id  should be considered as the primary key as both of them uniquely identify the rows.
#As part of Partition row store create and instert statments are made in the order of Primary key & Cluster key
# Select statment gives the Required out as per question 3.

query = "CREATE TABLE IF NOT EXISTS cassandra.User_song "
query = query + "(song_title text,userid int,first_name text,last_name text, PRIMARY KEY (song_title,userid))"
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO cassandra.User_song (song_title,userid,first_name,last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [18]:
query = "select first_name,last_name from User_song WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [19]:
Drop_query = "Drop TABLE cassandra.Session_event_details "
try:
    session.execute(Drop_query)
except Exception as e:
    print(e)

In [20]:
Drop_query = "Drop TABLE cassandra.User_Usage_hist "
try:
    session.execute(Drop_query)
except Exception as e:
    print(e)

In [21]:
Drop_query = "Drop TABLE cassandra.User_song "
try:
    session.execute(Drop_query)
except Exception as e:
    print(e)

In [22]:
session.shutdown()
cluster.shutdown()